In [ ]:
import torch

# 各部位的点
# hb:[0,3,6,7,8,9,12]
# rl:[0,3,4,5,6]
# ll:[0,1,2,3,6]
# lh:[7,9,10,11]
# rh:[7,12,13,14]
x = [0.0, 26.572193345670527, 94.63634589018879, 126.42089040290892, 26.589024107241325, 100.21257116017856, 133.42424869711368, 34.71791272997003,
     63.36697568392949, 99.34806581349444, 62.55397898136032, 89.34865683645273, 119.64919516674291, 64.76172065982712, 86.63553928432384, 117.37898708522131]

parts_info = {
    'hb': [0, 3, 6, 7, 8, 9, 12],
    'rl': [0, 3, 4, 5, 6],
    'll': [0, 1, 2, 3, 6],
    'lh': [7, 9, 10, 11],
    'rh': [7, 12, 13, 14],
}

print('16点分别mpjpe:')
print(x)
print(parts_info)

for key in parts_info:
    indexes = parts_info[key]
    res = []
    print(key)
    for index in indexes:
        res.append(x[index])
    print(res)
    res = torch.Tensor(res)
    res = torch.mean(res).item()
    print(res)


# 加载H36M数据


In [1]:
# 加载3D数据集
import os.path as path
from common.h36m_dataset import Human36mDataset
from utils.data_utils import read_3d_data
dataset_path=path.join('data','data_3d_'+'h36m'+'.npz')
dataset=Human36mDataset(dataset_path,True)
dataset=read_3d_data(dataset)

In [2]:
from utils.data_utils import create_2d_data

# 加载2D数据集
keypoints_path = path.join(
        'data', 'data_2d_' + 'h36m' + '_' + 'gt' + '.npz')
keypoints=create_2d_data(keypoints_path,dataset,tag='3dhp')

In [3]:
from utils.data_utils import fetch
import numpy as np
action_filter = None
stride = 1
subjects_train = ['S1', 'S5', 'S6', 'S7', 'S8']
subjects_test = ['S9', 'S11']

# train:600个数据的四种字段
poses_train, poses_train_2d, actions_train, cams_train = fetch(subjects_train, dataset, keypoints, action_filter,
                                                               stride)
# valid:236个数据的四种字段
poses_valid, poses_valid_2d, actions_valid, cams_valid = fetch(subjects_test, dataset, keypoints, action_filter,
                                                               stride)
poses_train, poses_train_2d, cams_train = np.concatenate(
        poses_train), np.concatenate(poses_train_2d), np.concatenate(cams_train)
poses_valid, poses_valid_2d, cams_valid = np.concatenate(
    poses_valid), np.concatenate(poses_valid_2d), np.concatenate(cams_valid)

In [4]:
from torch.utils.data import DataLoader
from common.data_loader import PoseDataSet
from functools import reduce

# prepare train loader for GT 2D - 3D, which will update by using projection.
pad = 13
batch_size = 1024
num_workers = 2
train_gt2d3d_loader = DataLoader(PoseDataSet(poses_train, poses_train_2d, actions_train, cams_train, pad),
                                 batch_size=batch_size,
                                 shuffle=True, num_workers=num_workers, pin_memory=True)

# GT 2D-3D的验证集副本
valid_loader = DataLoader(PoseDataSet(poses_valid, poses_valid_2d, actions_valid, cams_valid, pad),
                          batch_size=batch_size,
                          shuffle=False, num_workers=num_workers, pin_memory=True)
# 至此准备完事儿，可以拿来用

# 加载Target数据3dhp

In [5]:
from utils.data_utils import create_2d_data_target,fetch_target,fetch_3dhp_train
from common.data_loader import PoseTarget

keypoints_target = create_2d_data_target(path.join(
    'data', 'data_'+'3dhp' + '_' + 'gt' + '_train.npz'), dataset)
subjects_target = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7']
poses_target_2d = fetch_target(subjects_target, keypoints_target)
poses_target_2d = np.concatenate(poses_target_2d)
# poses_target_3d = fetch_3dhp_train(
#     path.join('data', 'data_'+'3dhp' + '_gt_train_3d.npz'))
target_2d_loader = DataLoader(PoseTarget(poses_target_2d),
                                  batch_size=batch_size,
                                  shuffle=True, num_workers=num_workers, pin_memory=True)

target_3d_loader = DataLoader(PoseTarget(poses_train),
                            batch_size=batch_size,
                            shuffle=True, num_workers=num_workers, pin_memory=True)

# just for visualization of camera distribution
# target_3d_loader2 = DataLoader(PoseTarget(poses_target_3d),
#                             batch_size=batch_size,
#                             shuffle=True, num_workers=num_workers, pin_memory=True)


Generating 1666994 poses...
Generating 1559752 poses...


# 使用数据

In [6]:
import torch
for i,((inputs_3d,_,_,cam_param),target_d2d, target_d3d) in enumerate(zip(train_gt2d3d_loader,target_2d_loader,target_3d_loader)):
    pad=(inputs_3d.shape[2]-1)//2
    print("pad: %d"%pad)
    # 所有输入值跟0判断，求和相当于数有几个0
    rows=torch.sum(inputs_3d==0,dim=(-1,-2,-3,-4))<(2*pad+1)*16*2
    inputs_3d=inputs_3d[rows] # 原分布数据
    cam_param=cam_param[rows] # 相机角度
    
    
    
    break

pad: 13
